Câu 1: Phân phối Bernoulli và Multinomial

Cho tập dữ liệu Education.csv [https://drive.google.com/file/d/1Gn6YWHXRuPbTUXY5HFxM5C_tJHuZxCka/view?usp=sharing]
- Trong đó:
    - Text: Chứa đoạn văn bản liên quan đến chủ đề giáo dục.
    - Label: Chứa nhãn cảm xúc của văn bản [Tích cực (Positive)/Tiêu cực (Negative)].
- Yêu cầu: Áp dụng thuật toán Naive Bayes (phân phối bernoulli và phân phối Multinomial) để dự đoán cảm xúc của văn bản là tích cực hay tiêu cực và so sánh kết quả của hai phân phối đó.

In [27]:
import pandas as pd
import re 
import nltk
import nltk
from nltk.corpus import stopwords
import random
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

[nltk_data] Downloading package punkt to /Users/hqhyy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/hqhyy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hqhyy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
data = pd.read_csv("Education.csv")
data.head()

,Text,Label
0,The impact of educational reforms remains unce...,positive
1,Critics argue that recent improvements in the ...,negative
2,Innovative teaching methods have led to unexpe...,positive
3,"Despite budget constraints, the school has man...",positive
4,The true effectiveness of online learning plat...,negative


In [29]:
def preprocess_text(text):
    text = text.lower()
    # Bỏ số
    text = re.sub(r'\d+', ' ', text)
    # Bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', '', text)
    # Convẻt String to Array
    tokens = nltk.word_tokenize(text)
    return tokens


#### Xoá từ dừng: "the", "is", "and"
![alt text](image-1.png)

In [30]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens 

#### Chuyển đổi từ thành dạng gốc:
![alt text](image.png)

In [31]:
def perform_lemmatization(tokens):
    lemmatization = nltk.WordNetLemmatizer()
    lemmatization_tokens = [lemmatization.lemmatize(token) for token in tokens]
    return lemmatization_tokens

In [32]:
def clean_text(text):
    tokens = preprocess_text(text)
    filtered_tokens = remove_stopwords(tokens)
    lemmatization_tokens = perform_lemmatization(filtered_tokens)
    clean_text = ' '.join(lemmatization_tokens)
    return clean_text

In [33]:
data.head()

,Text,Label
0,The impact of educational reforms remains unce...,positive
1,Critics argue that recent improvements in the ...,negative
2,Innovative teaching methods have led to unexpe...,positive
3,"Despite budget constraints, the school has man...",positive
4,The true effectiveness of online learning plat...,negative


In [34]:
data['Cleaned_Text'] = data['Text'].apply(clean_text)
data = data[['Cleaned_Text', 'Label']]

In [35]:
data.head()

,Cleaned_Text,Label
0,impact educational reform remains uncertain de...,positive
1,critic argue recent improvement school system ...,negative
2,innovative teaching method led unexpected chal...,positive
3,despite budget constraint school managed maint...,positive
4,true effectiveness online learning platform st...,negative


### Phân bố Dữ Liệu để train

In [36]:
text_train, text_test, labels_train, labels_test = train_test_split(
    data['Cleaned_Text'], 
    data['Label'], 
    test_size=0.2
    )

#### Train mô hình

In [37]:
vecto_hoa_van_ban_Bernoulli = CountVectorizer(binary=True)
vecto_hoa_van_ban_Multinomial = CountVectorizer()
model_1 = make_pipeline(vecto_hoa_van_ban_Bernoulli, BernoulliNB())
model_2 = make_pipeline(vecto_hoa_van_ban_Multinomial, MultinomialNB())
model_1.fit(text_train, labels_train)
model_2.fit(text_train, labels_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [38]:
predict_ber = model_1.predict(text_test)
predict_mul = model_2.predict(text_test)

#### So sánh 2 phân phối

In [39]:
print("Mo hinh Bernoulli:")
print(classification_report(labels_test, predict_ber))
print("Mo hinh Multinonial: ")
print(classification_report(labels_test, predict_mul))

Mo hinh Bernoulli:
              precision    recall  f1-score   support

    negative       0.40      1.00      0.57         4
    positive       1.00      0.14      0.25         7

    accuracy                           0.45        11
   macro avg       0.70      0.57      0.41        11
weighted avg       0.78      0.45      0.37        11

Mo hinh Multinonial: 
              precision    recall  f1-score   support

    negative       0.44      1.00      0.62         4
    positive       1.00      0.29      0.44         7

    accuracy                           0.55        11
   macro avg       0.72      0.64      0.53        11
weighted avg       0.80      0.55      0.51        11



#### Accuracy Score

In [40]:
print("Accuracy score of Bernoulli:", accuracy_score(labels_test, predict_ber))
print("Accuracy score of Multinonial:", accuracy_score(labels_test, predict_mul))

Accuracy score of Bernoulli: 0.45454545454545453
Accuracy score of Multinonial: 0.5454545454545454


In [52]:
a = "Budget cuts in education reduce teaching quality and negatively impact students."
a = clean_text(a)
result1 = model_1.predict([a])
result2 = model_2.predict([a])
print(result1, result2)

['negative'] ['negative']


Câu 2: Phân phối Gaussian

Cho tập dữ liệu Drug.csv [https://drive.google.com/file/d/1_G8oXkLlsauQkujZzJZJwibAWu5PgBXK/view?usp=sharing]
- Trong đó:
  - Age: Tuổi của bệnh nhân
  - Sex: Giới tính của bệnh nhân
  - BP: Mức huyết áp
  - Cholesterol: Mức cholesterol trong máu
  - Na_to_K: Tỷ lệ Natri và Kali trong máu
  - Drug: Loại thuốc [A/B/C/X/Y]
- Yêu cầu: Áp dụng thuật toán Naive Bayes (phân phối Gaussian) để dự đoán kết quả loại thuốc phù hợp với bệnh nhân.

Gaussian) để dự đoán kết quả loại thuốc phù hợp với bệnh nhân.

In [42]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

In [43]:
df = pd.read_csv("drug.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [44]:
df.shape

(200, 6)

In [45]:
x = df.drop("Drug", axis=1)
y = df['Drug']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
# print(X_train)

In [47]:
newX_train = X_train.copy()

le = LabelEncoder()
# Assuming 'Sex', 'BP', and 'Cholesterol' are categorical columns
newX_train['Sex'] = le.fit_transform(X_train['Sex'])
newX_train['BP'] = le.fit_transform(X_train['BP'])
newX_train['Cholesterol'] = le.fit_transform(X_train['Cholesterol'])
y = le.fit_transform(y)

In [48]:
model_3 = GaussianNB()
model_3.fit(newX_train, y_train)

GaussianNB()

In [49]:
newX_test = X_test.copy()

newX_test['Sex'] = le.fit_transform(X_test['Sex'])
newX_test['BP'] = le.fit_transform(X_test['BP'])
newX_test['Cholesterol'] = le.fit_transform(X_test['Cholesterol'])
predict_gaus = model_3.predict(newX_test)

In [50]:
print(accuracy_score(y_test, predict_gaus))

0.8
